In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Configurações do gráfico
plt.rcParams["figure.figsize"] = [15, 3]

# Função para ler dados RMSF
def read_rmsf_file(file_path):
    data = np.loadtxt(file_path, skiprows=1)
    return data[:, 0], data[:, 1]

# Caminhos base
path_CMY136 = "/user/work/ph23872/CMY-136-MC/MM-Trans-Analysis/RMSF/"
path_CMY2   = "/user/work/ph23872/CMY-2-MC/MM-Analysis/RMSF/"

# Lê dados das 5 réplicas de cada variante
replicas_CMY136 = []
replicas_CMY2 = []

#Ajuste aqui os nomes dos arquivos, se necessário. ATentar para renumeração das réplicas
for i in range(1, 6):
    _, rmsf_136 = read_rmsf_file(f"{path_CMY136}rmsf_CMY136.gauche.prod{i}.dat")
    _, rmsf_2   = read_rmsf_file(f"{path_CMY2}rmsf_CMY2.gauche.prod{i}.dat")
    replicas_CMY136.append(rmsf_136)
    replicas_CMY2.append(rmsf_2)

# Converte para arrays
rmsf_136 = np.vstack(replicas_CMY136)
rmsf_2   = np.vstack(replicas_CMY2)

# Calcula médias
mean_136 = np.mean(rmsf_136, axis=0)
mean_2   = np.mean(rmsf_2, axis=0)
delta_rmsf = mean_136 - mean_2

# Usa resíduos da CMY2 Replica 1 como referência
residues, _ = read_rmsf_file(f"{path_CMY2}rmsf_CMY2.gauche.prod1.dat")

# Filtra a partir do resíduo 16 (Ajuste se necessário para excluir largas flutuações do N ou C terminal)
mask = residues >= 16
residues = residues[mask]
delta_rmsf = delta_rmsf[mask]

# Criação do gráfico com ênfase em regiões importantes
for res, delta in zip(residues, delta_rmsf):
    color = '#DC143C' if 185 <= res <= 222 else '#708090'
    alpha = 1.0 if 185 <= res <= 222 else 0.3
    plt.bar(res, delta, color=color, alpha=alpha, width=0.8)

    # Adiciona rótulo se delta > 0.2 ou < -0.2
    if delta > 0.2 or delta < -0.2:
        plt.text(res, delta + 0.1*np.sign(delta), f"{int(res)}", 
                 ha='center', va='bottom' if delta > 0 else 'top',
                 fontsize=10, color='black', rotation=90)

# Estética
plt.axhline(0, color='black', linewidth=1)
plt.xlim(16, max(residues))
plt.xticks(size=12)
plt.yticks(size=12)
plt.xlabel("Residue Number", fontsize=14)
plt.ylabel("ΔRMSF$_{Cα}$ (Å)", fontsize=14)
plt.title("ΔRMSF: CMY-136 − CMY-2", fontsize=20, pad=20)

# Salvar se quiser
# plt.savefig("delta_RMSF_CMY136_vs_CMY2_labeled.png", dpi=600, bbox_inches="tight")

plt.show()
